In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
from io import BytesIO
from zipfile import ZipFile
import requests

# Descargar datos de precios de las acciones
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Puedes añadir más tickers
data = yf.download(tickers, start='2020-01-01', end='2023-01-01')['Adj Close']

# Calcular rendimientos diarios
returns = data.pct_change().dropna()

# Descargar y leer datos de factores Fama-French
ff_url = 'https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip'
response = requests.get(ff_url)
with ZipFile(BytesIO(response.content)) as z:
    with z.open(z.namelist()[0]) as f:
        ff_factors = pd.read_csv(f, skiprows=3)

# Filtrar filas no deseadas
ff_factors = ff_factors[ff_factors.iloc[:, 0].apply(lambda x: str(x).isdigit())]

# Renombrar columnas adecuadamente
ff_factors.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']
ff_factors['Date'] = pd.to_datetime(ff_factors['Date'], format='%Y%m%d')
ff_factors.set_index('Date', inplace=True)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RF']] / 100

# Alinear fechas
ff_factors = ff_factors.loc[returns.index]

# Calcular los excesos de retorno de las acciones
excess_returns = returns.subtract(ff_factors['RF'], axis=0)

# Preparar los factores Fama-French para la regresión
X = ff_factors[['Mkt-RF', 'SMB', 'HML']]
X = sm.add_constant(X)

# Diccionario para almacenar los resultados de las regresiones
results = {}

for ticker in tickers:
    y = excess_returns[ticker]
    model = sm.OLS(y, X).fit()
    results[ticker] = model

# Mostrar los resultados
for ticker, model in results.items():
    print(f'Resultados de la regresión para {ticker}:')
    print(model.summary())
    print('\n')

[*********************100%%**********************]  3 of 3 completed


Resultados de la regresión para AAPL:
                            OLS Regression Results                            
Dep. Variable:                   AAPL   R-squared:                       0.742
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     718.6
Date:                Tue, 21 May 2024   Prob (F-statistic):          3.63e-220
Time:                        22:06:57   Log-Likelihood:                 2279.4
No. Observations:                 755   AIC:                            -4551.
Df Residuals:                     751   BIC:                            -4532.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         